In [0]:
"""This is testing on real stars""";

In [0]:
"""
This program tests Affine+Ditrortion35 model
on real data (pairs of star coordinates)
""";

In [0]:
from pylab import *
from PIL import Image, ImageDraw
from functools import partial
import os

Get star pairs and calculate NUM_STAR_PAIRS (at least 5 because it's minimum needed for affine+distortion35 model)   

In [0]:
# whether to divide coordinates by SCALE_FAC 
# or not (zoomed coords or not)
# Увеличены ли координаты в файлах (увеличение х4 по умолчанию),
# полученных при ручной "сборке" пар звезд
zoomed_coords = True 
center_only = True # use only central stars \ Использовать только звезды из центральной области
SCALE_FAC = 4.0 # Scale factor of coordinates \ Коэффициент увеличения координат (x4 чаще всего)

In [0]:
"""
Load star coords from txt-files
""";

In [0]:
# folder with coords files \ Путь из корня проекта до папки с файлами координат
coords_folder = 'data/star_coords/2016nov-11_txt/' 
# Путь из корня проекта до папки с файлами изображений
images_folder = 'data/stars/2016nov-11/'

In [0]:
# 2016nov-11 jpg
# Массив имен файлов с координатами для использования
fnames = [
    "20161122-191517-359.txt",
    "20161121-220921-250.txt",
]

In [0]:
# Отбрасываем ".txt" (Берем "дату") из имени файла 
date = fnames[0].split('.')[0]

In [0]:
# Открываем изображение с префиксом "mod_" в имени, для получения размеров
im = Image.open(images_folder + "mod_" + date + "-1.jpg")
width, height = im.size
print("Image size:", width, height)

Image size: 3072 2304


In [0]:
# Находим координаты центра изображения
xCenter = width // 2
yCenter = height // 2
CENTER_RAD = 700 # radius(px) of central part \ Радиус центральной части
print('CENTER_RAD:', CENTER_RAD)

CENTER_RAD: 700


In [0]:
# Читаем все файлы с координатами и собираем все четверки координат (x1,y1,x2,y2) в массив четверок
coords_list = []
for fname in fnames:
    piece = np.loadtxt(coords_folder + os.sep + fname)
    coords_list.append(piece)

coords = np.vstack(coords_list)

In [0]:
# Если координаты масштабированы, то делим их на коэфф. масштабирования
if zoomed_coords:
    coords /= float(SCALE_FAC)
    coords = coords.round()
    print('Normal Star coordinates pairs (first 5):\n', coords[:5], '\n')


Normal Star coordinates pairs (first 5):
 [[ 227.  418.  531.  473.]
 [ 199.  681.  508.  733.]
 [ 378.  781.  684.  830.]
 [1310.  305. 1606.  342.]
 [1397. 1180. 1706. 1225.]] 



In [0]:
# Если берем только центральные звезды, то отбрасываем все, не попадающие в пересечение 2х окружностей (левого и прав изображ-й)
if center_only:
    coords_center = []
    
    for i in range(coords.shape[0]):
        _lx = coords[i, 0]
        _ly = coords[i, 1]
        _rx = coords[i, 2]
        _ry = coords[i, 3]
        if \
        (_lx - xCenter)**2 + (_ly - yCenter)**2 <= CENTER_RAD**2 and \
        (_rx - xCenter)**2 + (_ry - yCenter)**2 <= CENTER_RAD**2:
            coords_center.append(coords[i])
    
    coords = np.vstack(coords_center)
    
    print('Normal Star coordinates pairs in center:\n', coords[:5], '\n')

Normal Star coordinates pairs in center:
 [[1397. 1180. 1706. 1225.]
 [1138. 1124. 1442. 1170.]
 [1397. 1180. 1706. 1225.]
 [1138. 1124. 1442. 1170.]] 



In [0]:
# Число отобранных пар звезд
NUM_STAR_PAIRS = len(coords)
N = NUM_STAR_PAIRS
# Количество координат в "четверке" {lX, lY, rX, rY}
M = coords.shape[1] # {lX, lY, rX, rY} == 4
print('Number of Star coordinates pairs:', NUM_STAR_PAIRS)

Number of Star coordinates pairs: 4


In [0]:
leftX = coords[:, 0] # Левые Х координаты
leftY = coords[:, 1] # Левые У координаты
rightX = coords[:, 2] # Правые Х координаты
rightY = coords[:, 3] # Правые У координаты

print('''First 5 pairs
Left X: {}
Left Y: {}
'''.format(leftX[:5], leftY[:5])
)
print('''\
Right X: {}
Right Y: {}
'''.format(rightX[:5], rightY[:5])
)

First 5 pairs
Left X: [1397. 1138. 1397. 1138.]
Left Y: [1180. 1124. 1180. 1124.]

Right X: [1706. 1442. 1706. 1442.]
Right Y: [1225. 1170. 1225. 1170.]



In [0]:
ELL_RAD = 3 # Радиус круга звезды для отрисовки схемы расположения свезд

# Draw star pairs \ Создаем Новое изображение
scatterOriginal = Image.new('RGB', (width, height), 'lightgray')
# Инструмент для рисования
draw = ImageDraw.Draw(scatterOriginal)

# Central point \ Рисуем центральную точку
draw.ellipse((xCenter - ELL_RAD, yCenter - ELL_RAD, 
              xCenter + ELL_RAD, yCenter + ELL_RAD), fill='darkgreen')

# Draw central part boundary \ Рисуем центральную окружность
draw.ellipse((xCenter - CENTER_RAD, yCenter - CENTER_RAD, 
              xCenter + CENTER_RAD, yCenter + CENTER_RAD), outline='black')


# Рисуем круги-звезды левого изображения
for i in range(NUM_STAR_PAIRS): # draw star points
    draw.ellipse((leftX[i] - ELL_RAD, leftY[i] - ELL_RAD, 
                  leftX[i] + ELL_RAD, leftY[i] + ELL_RAD), fill='blue')


In [0]:
# Сохраняем изображение на диск
scatterOriginal.save('orig.png')

affine coeffincients  
(a,b,  
 c,d) -- for rotation matrix  
(e,f) -- for transition (shift)   

In [0]:
# Функция для аффинной трансформации координат х,у
# Принимает: xy -- tuple из координат х,у: (х,у)
# coeffs -- tuple из коэффициентов преобразования: (a,b,c,d,e,f)
# Возвращает: [x, y] -- трансформированные координаты в виде массива из 2х элементов: [х,у]
def affine_transform(xy, coeffs=(1,0,0,1,0,0)):
    assert coeffs != (1,0,0,1,0,0)
        
    _a, _b, _c, _d, _e, _f = coeffs
    x, y = xy
    return [
        _a * x + _b * y + _e,
        _c * x + _d * y + _f
    ]

In [0]:
# Записываем исходные координаты в другие переменные, т.к. массивы
# leftX, leftY, rightX, rightY будут использованы для записи в них съюстированных координат
# inputLeftX, inputLeftY, inputRightX, inputRightY are coordinates we get from measuring system
inputLeftX = leftX
inputLeftY = leftY

inputRightX = rightX
inputRightY = rightY

In [0]:
# Функция для коррекции дисторсии координат х,у
# Принимает: xy -- tuple из координат х,у: (х,у)
# coeffs -- tuple из коэффициентов преобразования: (eps1_or_eps2, eps3_or_eps4)
# Возвращает: [x, y] -- трансформированные координаты в виде массива из 2х элементов: [х,у]
def correct_distort(xy, coeffs=(0,0)):
    assert coeffs != (0,0)
    
    # eps1, eps3 -- for left img
    # eps2, eps4 -- for right img
    _eps1_or_eps2, _eps3_or_eps4  = coeffs
    
    x, y = xy
    
    # squared distance from center to (x, y) point
    _r = (x - xCenter) ** 2 + (y - yCenter) ** 2
    
    return [
        x - (x - xCenter) * ( _r * _eps1_or_eps2 + (_r ** 2) * _eps3_or_eps4 ),
        y - (y - yCenter) * ( _r * _eps1_or_eps2 + (_r ** 2) * _eps3_or_eps4 )
    ]

### Test affine model

Calculate model coefficients

In [0]:
# Инициализируем leftX, leftY, rightX, rightY измеренными значениями
leftX = inputLeftX
leftY = inputLeftY
rightX = inputRightX
rightY = inputRightY

In [0]:
# Заполняем вектор кси (из ИВС схемы измерения $ A * \xi = z + \nu$ )
xi = np.zeros(2 * NUM_STAR_PAIRS)

for i in range(NUM_STAR_PAIRS): # fill the xi vector
    xi[2 * i] = rightX[i]
    xi[2 * i + 1] = rightY[i]

print('xi (first 5):\n', xi[:5])

xi (first 5):
 [1706. 1225. 1442. 1170. 1706.]


In [0]:
k = 6 # num of coeff-s \ Число коэфф-ов для аффинной модели

z = np.zeros(k) # Создаем вектор оцениваемых параметров z
arr = np.zeros((2 * NUM_STAR_PAIRS, k)) # matrix A \ Создаем матрицу А

for i in range(NUM_STAR_PAIRS): # fill the A matrix \ Заполняем матрицу А
    
    arr[2 * i] = [leftX[i], leftY[i], 0, 0, 1, 0]

    arr[2 * i + 1] = [0, 0, leftX[i], leftY[i], 0, 1]

    
p_arr = pinv(arr, rcond=1e-20) # Считаем псевдо-обратную матрицу А^-
z = np.dot(p_arr, xi) # Считаем вектор z = A^- * xi

print("""
Affine coefficients:
%.4f %.4f %.4f %.4f 
%.2f %.2f""" % tuple(z))
print('cond(A): ', np.linalg.cond(arr))


Affine coefficients:
0.9498 0.3212 0.7656 -2.3750 
0.00 3672.00
cond(A):  inf


/home/stan/anaconda3/lib/python3.6/site-packages/numpy/linalg/linalg.py:1543: RuntimeWarning: divide by zero encountered in true_divide
  return s[..., 0]/s[..., -1]


In [0]:
"""
Align images and blend

a) Affine
""";

In [0]:
# Создаем функцию из функции affine_transform, зафиксировав в ней coeffs
affine = partial(affine_transform, coeffs=tuple(z))

# Calc estimated (affine transformed) points \ Применяем (map) affine к "склеенным попарно" массивам leftX, leftY ( zip(leftX, leftY) ) 
# Преобразуем к массиву [[х0, x1 ... ] [y0, y1, ...]]
leftCoords = array(list(map(affine, zip(leftX, leftY))))

# Estimated coordinates \ Вытаскиваем съюстированные координаты звезд левого изображения
estLeftX = leftCoords[:, 0]
estLeftY = leftCoords[:, 1]


print('''Backward affine Left:
Left X: {}
Left Y: {}
'''.format(estLeftX[:5], estLeftY[:5])
)

print('''Right:
Right X: {}
Right Y: {}
'''.format(rightX[:5], rightY[:5])
)

Backward affine Left:
Left X: [1706. 1442. 1706. 1442.]
Left Y: [1939.078125 1873.78125  1939.078125 1873.78125 ]

Right:
Right X: [1706. 1442. 1706. 1442.]
Right Y: [1225. 1170. 1225. 1170.]



Calculate error metrics

1) $\Delta x_i, \Delta y_i, \; i = 1,N$

2) $\sigma^2 = \frac{1}{N} \sum\limits_{i=1}^{N} 
                \left( \Delta x_i^2 + \Delta y_i^2 \right)$

In [0]:
# Считаем метрики, описанные формулами сверху
delX = abs(estLeftX - rightX)
delY = abs(estLeftY - rightY)
print("delX:", delX[:5])
print("delY:", delY[:5])

sigSqr = 1.0 / N * sum(delX**2 + delY**2)
mX = max(delX)
mY = max(delY)
m = max(mX, mY)

print("mX: %.4f mY: %.4f m: %.4f" % (mX, mY, m))
print("sigSqr: %.4f" % sigSqr)

delX: [1.36424205e-12 1.13686838e-12 1.36424205e-12 1.13686838e-12]
delY: [714.078125 703.78125  714.078125 703.78125 ]
mX: 0.0000 mY: 714.0781 m: 714.0781
sigSqr: 502607.8082


Plot aligned star pairs

In [0]:
# Рисуем съюстированные звезды, см комменты в аналогичном блоке выше
scatter = Image.new('RGB', (width, height), 'lightgray')


draw = ImageDraw.Draw(scatter)
draw.ellipse((xCenter - ELL_RAD, yCenter - ELL_RAD, 
              xCenter + ELL_RAD, yCenter + ELL_RAD), fill='darkgreen')


# draw star points \ Рисуем совмещенные  левые (red) и правые (blue) звезды
for i in range(NUM_STAR_PAIRS):
    draw.ellipse((estLeftX[i] - ELL_RAD, estLeftY[i] - ELL_RAD, 
                  estLeftX[i] + ELL_RAD, estLeftY[i] + ELL_RAD), fill='red')
    
    draw.ellipse((rightX[i] - ELL_RAD, rightY[i] - ELL_RAD, 
                  rightX[i] + ELL_RAD, rightY[i] + ELL_RAD), fill='blue')

In [0]:
scatter.save('000.png')

### Test affine+distortion35 model

Calculate model coefficients

In [0]:
leftX = inputLeftX
leftY = inputLeftY
rightX = inputRightX
rightY = inputRightY

In [0]:
"""
c) Affine + Ditortion 3rd, 5th orders 
  (at least 5 stars)
""";

In [0]:
k35 = 10 # Число коэфф-ов для модели D35

z35 = np.zeros(k35) # искомый вектор параметров z
arr35 = np.zeros((2 * N, k35)) # matrix A

for i in range(N): # fill the A matrix \ Заполняем матрицу А
    # Квадрат расстояния от точки до центра на левом изображении
    dist_l = (leftX[i] - xCenter) ** 2 + (leftY[i] - yCenter) ** 2
    # Квадрат расстояния от точки до центра на правом изображении
    dist_r = (rightX[i] - xCenter) ** 2 + (rightY[i] - yCenter) ** 2

    zx1 = (leftX[i] - xCenter) * dist_l # (х * r^2) на левом кадре
    zx2 = (rightX[i] - xCenter) * dist_r # (x * r^2) на правом кадре
    wx1 = (leftX[i] - xCenter) * dist_l ** 2 # (х * r^4) на левом кадре
    wx2 = (rightX[i] - xCenter) * dist_r ** 2 # (x * r^4) на правом кадре

    # Четные (0, 2, ...) строки матрицы А
    arr35[2 * i] = [leftX[i], leftY[i], 0, 0, 1, 0, -zx1, zx2, -wx1, wx2]

    # Аналогичные элементы для у-координаты     
    zy1 = (leftY[i] - yCenter) * dist_l
    zy2 = (rightY[i] - yCenter) * dist_r
    wy1 = (leftY[i] - yCenter) * dist_l ** 2
    wy2 = (rightY[i] - yCenter) * dist_r ** 2

    # Нечетные (1, 3, ...) строки матрицы А
    arr35[2 * i + 1] = [0, 0, leftX[i], leftY[i], 0, 1, -zy1, zy2, -wy1, wy2]


In [0]:
p_arr35 = pinv(arr35, rcond=1e-20) # Считаем псевдообратную матрицу A^-
z35 = np.dot(p_arr35, xi) # Считаем вектор параметров z = A^- * xi


print("""
Affine coefficients + Ditortion 3rd, 5th orders:

%.4f %.4f %.4f %.4f 
%.2f %.2f 
%.2e %.2e 
%.2e %.2e""" % tuple(z35))

print('cond(A): ', np.linalg.cond(arr35))


Affine coefficients + Ditortion 3rd, 5th orders:

-0.0000 -0.0000 0.0000 0.0000 
-0.00 0.00 
-3.32e-04 6.31e-03 
2.63e-09 -1.72e-07
cond(A):  2.693263182469305e+24


In [0]:
"""
c) Affine + Ditortion3,5
""";

In [0]:
a = float(z35[0])
b = float(z35[1])
c = float(z35[2])
d = float(z35[3])
e = float(z35[4])
f = float(z35[5])

eps1 = float(z35[6])
eps2 = float(z35[7])
eps3 = float(z35[8])
eps4 = float(z35[9])

In [0]:
# Backward distort
correctDistortLeft = partial(correct_distort, coeffs=(eps1, eps3))
# Применяем дисторсионные поправки к лев. звездам
leftCoords = array(list(map(correctDistortLeft, zip(leftX, leftY))))
leftX = leftCoords[:, 0]
leftY = leftCoords[:, 1]


correctDistortRight = partial(correct_distort, coeffs=(eps2, eps4))
# Применяем дисторсионные поправки к прав. звездам
rightCoords = array(list(map(correctDistortRight, zip(rightX, rightY))))
estRightX35 = rightCoords[:, 0]
estRightY35 = rightCoords[:, 1]


# Backward affine
affine = partial(affine_transform, coeffs=(a,b,c,d,e,f))
# Применяем афф. преобразование к лев. звездам
leftCoords = array(list(map(affine, zip(leftX, leftY))))
estLeftX35 = leftCoords[:, 0]
estLeftY35 = leftCoords[:, 1]


print('''Backward distort+affine Left:
Left X: {}
Left Y: {}
'''.format(estLeftX35[:5], estLeftY35[:5])
)

print('''Backward distort Right:
Right X: {}
Right Y: {}
'''.format(estRightX35[:5], estRightY35[:5])
)

Backward distort+affine Left:
Left X: [-0.00023265 -0.00141931 -0.00023265 -0.00141931]
Left Y: [0.08331201 0.35032647 0.08331201 0.35032647]

Backward distort Right:
Right X: [-780.39626804 5522.55120006 -780.39626804 5522.55120006]
Right Y: [157.31219078 388.61785531 157.31219078 388.61785531]



Calculate error metrics

In [0]:
# Считаем метрики: dx, dy, sigma^2
delX35 = abs(estLeftX35 - estRightX35)
delY35 = abs(estLeftY35 - estRightY35)
print("delX35:", delX35[:5])
print("delY35:", delY35[:5])

sigSqr35 = 1.0 / N * sum(delX35**2 + delY35**2)
mX35 = max(delX35)
mY35 = max(delY35)
m35 = max(mX35, mY35)

print("mX35: %.4f mY35: %.4f m35: %.4f" % (mX35, mY35, m35))
print("sigSqr35: %.4f" % sigSqr35)

delX35: [ 780.39603538 5522.55261938  780.39603538 5522.55261938]
delY35: [157.22887877 388.26752884 157.22887877 388.26752884]
mX35: 5522.5526 mY35: 388.2675 m35: 5522.5526
sigSqr35: 15641539.0000


Plot aligned star pairs

In [0]:
# Рисуем съюстированные звезды
scatter35 = Image.new('RGB', (width, height), 'lightgray')


draw = ImageDraw.Draw(scatter35)
draw.ellipse((xCenter - ELL_RAD, yCenter - ELL_RAD, 
              xCenter + ELL_RAD, yCenter + ELL_RAD), fill='darkgreen')


for i in range(NUM_STAR_PAIRS): # draw star points / Рисуем совмещенные  левые (red) и правые (blue) звезды
    draw.ellipse((estLeftX35[i] - ELL_RAD, estLeftY35[i] - ELL_RAD, 
                  estLeftX35[i] + ELL_RAD, estLeftY35[i] + ELL_RAD), fill='red')
    
    draw.ellipse((estRightX35[i] - ELL_RAD, estRightY35[i] - ELL_RAD, 
                  estRightX35[i] + ELL_RAD, estRightY35[i] + ELL_RAD), fill='blue')


In [0]:
scatter35.save('035.png') # Сохраняем рисунок